In [2]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import pickle as pkl
import spacy
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, f_classif
import matplotlib.pyplot as plt

In [13]:
file_list_1 = os.listdir() 
file_list_1 # shows you the content of a folder


['EP_Heartbreak Hotel [First "Stand Up" Show] [Live].txt',
 'EP_Baby, What You Want Me to Do [First "Stand Up" Show] [Live].txt',
 "MJ_It's Too Late to Change the Time.txt",
 'MJ_Tabloid Junkie.txt',
 'EP_Blue Christmas.txt',
 'MJ_Childhood [Theme from "Free Willy 2"].txt',
 'EP_Suspicious Minds.txt',
 '4th_project_text_classification_1-Copy2.ipynb',
 "EP_Jailhouse Rock [First 'Stand-Up' Show].txt",
 'MJ_Earth Song.txt',
 'EP_Jailhouse Rock [Second "Stand Up" Show] [Live].txt',
 'EP_I Got Stung.txt',
 'MJ_Working Day and Night.txt',
 'MJ_Lovely One.txt',
 "EP_Blue Suede Shoes [Second 'Sit-Down' Show].txt",
 'EP_One Night [Second "Sit Down" Show] [Live].txt',
 'Text_classification_model.ipynb',
 "MJ_She's Out of My Life.txt",
 "EP_She's Not You.txt",
 'EP_A Little Less Conversation.txt',
 'MJ_Black or White.txt',
 "MJ_Don't Matter to Me.txt",
 'EP_I Feel So Bad.txt',
 'MJ_Bad.txt',
 'EP_If I Can Dream [From the NBC Elvis All-Star Tribute].txt',
 'MJ_Heal the World.txt',
 'web scraping_w

Create X and y

In [14]:
pre_X_only_EP = []
for file_name in file_list_1:
    if file_name.startswith('EP'):
        with open(file_name, 'r') as lyrics_file_1:
            pre_X_only_EP.append(lyrics_file_1.read())

In [15]:
pre_X_only_MJ = []
for file_name in file_list_1:
    if file_name.startswith('MJ'):
        with open(file_name, 'r') as lyrics_file_2:
            pre_X_only_MJ.append(lyrics_file_2.read())

The CountVectorizer expects a list of strs

In [16]:
pre_X_only_EP 

["Well, since my baby left me\nWell, I found a new place to dwell\nWell, it's down at the end of Lonely Street\nAt Heartbreak Hotel\n\nWell, I'll be\nI'll be so lonely baby\nWell, I'm so lonely\nI'll be so lonely, I could die\n\nOh, although it's always crowded\nYou still can find some room\nFor broken hearted lovers\nTo cry there in their gloom\n\nThey'll be so\nThey'll be so lonely, baby\nWell, they're so lonely\nThey're so lonely, they could die\n\nNow, the bell hop's tears keep flowin'\nAnd the desk clerk's dressed in black\nWell, they been so long on Lonely Street\nThey'll never ever look back\n\nAnd it's so\nWell, it's so lonely baby\nWell, they're so lonely\nWell, they're so lonely, they could've die\n\nWell, if your baby leaves you\nYou got a tale to tell\nWell, just take a walk down Lonely Street\nTo Heartbreak Hotel\n\nWhere you will be\nYou'll be so lonely, baby\nWell you'll be lonely\nYou'll be so lonely you could die\n\nOh, although it's always crowded\nYou still can find 

In [18]:
len(pre_X_only_EP)

48

In [7]:
pre_X_only_MJ 

["Too Late \nToo Late \nToo Late \nIn this almighty cage \nWe shop through the yellow page \nGone is the corner store \nThere is a supermarket there for you \n\nBut feelings remain the same \n\nAnd it's too late, too late to change the time \nWell it's too late, too late to change the time \nAnd it's too late, too late to change the time \nBut it's not too late to change your mind (too late) \n\nIt's a new day, a new age \nA photograph is taken, its ready one minute later \nCars are bigger and better \nManufacture satisfy\nAny world full of crime \nI can't let you take\n\nAnd it's too late, too late to change the time \nAnd it's too late, too late to change the time \nAnd it's too late, too late to change the time \nBut it's not too late to change your mind (too late) \nNo, no \n\nSo you want to be a star \nI think you are \nYou don't have to go further \nThen my loving arms \n\nAnd it's too late, too late to change the time \nBut it's not too late (too late) to change your mind \n\nAr

In [20]:
pre_X = pre_X_only_EP.copy()

In [21]:
pre_X.extend(pre_X_only_MJ )

In [9]:
print(pre_X_only_EP)

["Well, since my baby left me\nWell, I found a new place to dwell\nWell, it's down at the end of Lonely Street\nAt Heartbreak Hotel\n\nWell, I'll be\nI'll be so lonely baby\nWell, I'm so lonely\nI'll be so lonely, I could die\n\nOh, although it's always crowded\nYou still can find some room\nFor broken hearted lovers\nTo cry there in their gloom\n\nThey'll be so\nThey'll be so lonely, baby\nWell, they're so lonely\nThey're so lonely, they could die\n\nNow, the bell hop's tears keep flowin'\nAnd the desk clerk's dressed in black\nWell, they been so long on Lonely Street\nThey'll never ever look back\n\nAnd it's so\nWell, it's so lonely baby\nWell, they're so lonely\nWell, they're so lonely, they could've die\n\nWell, if your baby leaves you\nYou got a tale to tell\nWell, just take a walk down Lonely Street\nTo Heartbreak Hotel\n\nWhere you will be\nYou'll be so lonely, baby\nWell you'll be lonely\nYou'll be so lonely you could die\n\nOh, although it's always crowded\nYou still can find 

The CountVectorizer expects a list of strs

In [22]:
len(pre_X)

88

In [23]:
cv = CountVectorizer(lowercase=True, stop_words='english', token_pattern='[A-Za-z]+', ngram_range=(1,1))

In [24]:
X = cv.fit_transform(pre_X)

In [25]:
X  # this is a sparse matrix - it's more efficient to store only the non-zero elements

<88x1582 sparse matrix of type '<class 'numpy.int64'>'
	with 4401 stored elements in Compressed Sparse Row format>

In [26]:
print(X);  # here you see the index of the non-zero elements 

  (0, 62)	7
  (0, 741)	1
  (0, 886)	1
  (0, 950)	1
  (0, 403)	1
  (0, 1080)	7
  (0, 418)	1
  (0, 772)	18
  (0, 1300)	3
  (0, 600)	2
  (0, 644)	2
  (0, 769)	10
  (0, 801)	1
  (0, 358)	5
  (0, 902)	2
  (0, 299)	2
  (0, 1064)	2
  (0, 154)	2
  (0, 601)	2
  (0, 788)	2
  (0, 533)	2
  (0, 94)	1
  (0, 636)	1
  (0, 1372)	1
  (0, 483)	1
  :	:
  (87, 469)	7
  (87, 64)	2
  (87, 99)	6
  (87, 117)	1
  (87, 1074)	1
  (87, 1413)	1
  (87, 1026)	2
  (87, 1448)	1
  (87, 1506)	7
  (87, 1556)	1
  (87, 322)	1
  (87, 824)	7
  (87, 1106)	1
  (87, 1427)	1
  (87, 73)	49
  (87, 238)	1
  (87, 440)	1
  (87, 694)	1
  (87, 363)	1
  (87, 803)	1
  (87, 341)	7
  (87, 1175)	7
  (87, 511)	7
  (87, 370)	7
  (87, 957)	1


In [27]:
X.todense()  # turning a sparse matrix into a standard array

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [28]:
y = [0] * 48 + [1] * 40  # 48 songs of Elvis Presley and 4osongs of Michael Jackson

Song lines as samples
1.Split songs into lines

# MODEL

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [35]:
X_train.shape

(66, 1582)

In [30]:
model = LogisticRegression()

In [31]:
model.fit(X_train, y_train)

LogisticRegression()

In [32]:
model.score(X_train, y_train)

1.0

In [33]:
model.score(X_test, y_test)

0.8636363636363636